<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Description" data-toc-modified-id="Data-Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Description</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Useful-Scripts" data-toc-modified-id="Useful-Scripts-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Useful Scripts</a></span></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load the data</a></span></li><li><span><a href="#Data-Processing-and-Feature-Engineering" data-toc-modified-id="Data-Processing-and-Feature-Engineering-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Data Processing and Feature Engineering</a></span><ul class="toc-item"><li><span><a href="#missing-values" data-toc-modified-id="missing-values-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>missing values</a></span></li><li><span><a href="#size-reduction" data-toc-modified-id="size-reduction-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>size reduction</a></span></li><li><span><a href="#Take-date-of-only-2016" data-toc-modified-id="Take-date-of-only-2016-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Take date of only 2016</a></span></li><li><span><a href="#Melt-the-data" data-toc-modified-id="Melt-the-data-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Melt the data</a></span></li><li><span><a href="#Create-date-time-columns" data-toc-modified-id="Create-date-time-columns-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Create date time columns</a></span></li><li><span><a href="#Create-aggregation-columns" data-toc-modified-id="Create-aggregation-columns-5.6"><span class="toc-item-num">5.6&nbsp;&nbsp;</span>Create aggregation columns</a></span></li></ul></li><li><span><a href="#String-Manipulation" data-toc-modified-id="String-Manipulation-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>String Manipulation</a></span></li><li><span><a href="#Save-the-clean-data" data-toc-modified-id="Save-the-clean-data-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Save the clean data</a></span></li></ul></div>

# Data Description
Reference: https://www.kaggle.com/c/web-traffic-time-series-forecasting/data

train_1.csv:
```
rows = 145,063
columns = 551

first column = Page	
date columns = 2015-07-01, 2015-07-02, ..., 2016-12-31 (550 columns)

file size: 284.6 MB



Date columns:
------------------
Jul/2015 - 31 days
Aug/2015 - 31 days
Sep/2015 - 30 days
Oct/2015 - 31 days
Nov/2015 - 30 days
Dec/2015 - 31 days

Total     : 184 days
Year 2016 : 366 days (leap year)
Total     : 550 days

NOTE:
For this dataset, missing data is represented by 0.

```

The training dataset consists of approximately 145k time series. Each of these time series represent a number of daily views of a different Wikipedia article, starting from July, 1st, 2015 up until December 31st, 2016. The leaderboard during the training stage is based on traffic from January, 1st, 2017 up until March 1st, 2017.

The second stage will use training data up until September 1st, 2017. The final ranking of the competition will be based on predictions of daily views between September 13th, 2017 and November 13th, 2017 for each article in the dataset. You will submit your forecasts for these dates by September 12th.

For each time series, you are provided the name of the article as well as the type of traffic that this time series represent (all, mobile, desktop, spider). You may use this metadata and any other publicly available data to make predictions. Unfortunately, the data source for this dataset does not distinguish between traffic values of zero and missing values. A missing value may mean the traffic was zero or that the data is not available for that day.

To reduce the submission file size, each page and date combination has been given a shorter Id. The mapping between page names and the submission Id is given in the key files.


# Imports

In [37]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

import os
import time

# random state
random_state=100
np.random.seed(random_state) # we need this in each cell
np.random.set_state=random_state

# Jupyter notebook settings for pandas
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', '{:,.2g}'.format) # numbers sep by comma
pd.set_option('display.max_rows', 100) # None for all the rows
pd.set_option('display.max_colwidth', 100)

import IPython
from IPython.display import display

print([(x.__name__,x.__version__) for x in [np, pd,sns,matplotlib]])

[('numpy', '1.16.4'), ('pandas', '0.25.0'), ('seaborn', '0.9.0'), ('matplotlib', '3.1.1')]


In [3]:
import re

In [4]:
import dask
import dask.dataframe as dd
import gc

In [5]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

# Useful Scripts

In [6]:
def show_method_attributes(method, ncols=7):
    """ Show all the attributes of a given method.
    Example:
    ========
    show_method_attributes(list)
     """
    x = [i for i in dir(method) if i[0].islower()]
    x = [i for i in x if i not in 'os np pd sys time psycopg2'.split()]

    return pd.DataFrame(np.array_split(x,ncols)).T.fillna('')

In [7]:
def json_dump_tofile(myjson,ofile,sort_keys=False):
    """Write json dictionary to a datafile.
    
    Usage:
    myjson = {'num': 5, my_list = [1,2,'apple']}
    json_dump_tofile(myjson, ofile)
    
    """
    import io
    import json

    with io.open(ofile, 'w', encoding='utf8') as fo:
        json_str = json.dumps(myjson,
                          indent=4,
                          sort_keys=sort_keys,
                          separators=(',', ': '),
                          ensure_ascii=False)
        fo.write(str(json_str))

# Load the data

In [9]:
train = pd.read_csv('../../data/wiki/train_1.csv')
print(train.shape)
train.head()

(145063, 551)


,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,2015-07-11,2015-07-12,2015-07-13,2015-07-14,2015-07-15,2015-07-16,2015-07-17,2015-07-18,2015-07-19,2015-07-20,2015-07-21,2015-07-22,2015-07-23,2015-07-24,2015-07-25,2015-07-26,2015-07-27,2015-07-28,2015-07-29,2015-07-30,2015-07-31,2015-08-01,2015-08-02,2015-08-03,2015-08-04,2015-08-05,2015-08-06,2015-08-07,2015-08-08,2015-08-09,2015-08-10,2015-08-11,2015-08-12,2015-08-13,2015-08-14,2015-08-15,2015-08-16,2015-08-17,2015-08-18,...,2016-11-12,2016-11-13,2016-11-14,2016-11-15,2016-11-16,2016-11-17,2016-11-18,2016-11-19,2016-11-20,2016-11-21,2016-11-22,2016-11-23,2016-11-24,2016-11-25,2016-11-26,2016-11-27,2016-11-28,2016-11-29,2016-11-30,2016-12-01,2016-12-02,2016-12-03,2016-12-04,2016-12-05,2016-12-06,2016-12-07,2016-12-08,2016-12-09,2016-12-10,2016-12-11,2016-12-12,2016-12-13,2016-12-14,2016-12-15,2016-12-16,2016-12-17,2016-12-18,2016-12-19,2016-12-20,2016-12-21,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18,11,5,13,14,9,9,22,26,24,19,10,14,15,8,16,8,8,16,7,11,10,20,18,15,14,49,10,16,18,8,5,9,7,13,9,7,4,11,10,5,9,9,9,9,13,4,15,25,...,13,8,15,14,12,6,11,10,42,21,24,14,11,2e+02,14,45,33,28,18,14,47,15,14,18,20,14,16,14,20,60,22,15,17,19,18,21,21,47,65,17,32,63,15,26,14,20,22,19,18,20
1,2PM_zh.wikipedia.org_all-access_spider,11,14,15,18,11,13,22,11,10,4,41,65,57,38,20,62,44,15,10,47,24,17,22,9,39,13,11,12,21,19,9,15,33,8,8,7,13,2,23,12,27,27,36,23,58,80,60,69,42,...,12,11,14,28,23,20,9,12,11,14,14,15,15,11,20,13,19,6.2e+02,57,17,23,19,21,47,28,22,22,65,27,17,17,13,9,18,22,17,15,22,23,19,17,42,28,15,9,30,52,45,26,20
2,3C_zh.wikipedia.org_all-access_spider,1,0,1,1,0,4,0,3,4,4,1,1,1,6,8,6,4,5,1,2,3,8,8,6,6,2,2,3,2,4,3,3,5,3,5,4,2,5,1,4,5,0,0,7,3,5,1,6,2,...,6,4,2,4,6,5,4,4,3,3,9,3,5,4,0,1,4,5,8,8,1,1,2,5,3,3,3,7,3,9,8,3,2.1e+02,5,4,6,2,2,4,3,3,1,1,7,4,4,6,3,4,17
3,4minute_zh.wikipedia.org_all-access_spider,35,13,10,94,4,26,14,9,11,16,16,11,23,1.4e+02,14,17,85,4,30,22,9,10,11,7,7,11,9,11,44,8,14,19,10,17,17,10,7,10,1,8,27,19,16,2,84,22,14,47,25,...,38,13,14,17,26,14,10,9,23,15,7,10,7,10,14,17,11,9,11,5,10,8,17,13,23,40,16,17,41,17,8,9,18,12,12,18,13,18,23,10,32,10,26,27,16,11,17,19,10,11
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,13,11,8,6,10,14,6,9,6,16,14,13,15,14,16,9,1.8e+02,64,12,10,11,6,8,7,9,8,5,11,8,4,15,5,8,8,6,7,15,4,11,7,48,9,25,13,3,11,27,13,36,10


# Data Processing and Feature Engineering

## missing values

In [10]:
# description said zeros and nans are same
train = train.fillna(0)

## size reduction

In [11]:
train.memory_usage(deep=True).sum() * 1e-6 # MB

658.502795

In [12]:
train.iloc[:,1:].max().max()

67264258.0

In [13]:
train.dtypes

Page           object
2015-07-01    float64
2015-07-02    float64
2015-07-03    float64
2015-07-04    float64
               ...   
2016-12-27    float64
2016-12-28    float64
2016-12-29    float64
2016-12-30    float64
2016-12-31    float64
Length: 551, dtype: object

In [14]:
np.iinfo(np.int32).max

2147483647

In [15]:
np.iinfo(np.int32).max > 67264258.0

True

In [16]:
# I can use int32 as datatype

In [17]:
%%time
train.iloc[:,1:] = train.iloc[:,1:].astype(np.int32)

CPU times: user 50.3 s, sys: 45.2 s, total: 1min 35s
Wall time: 1min 48s


In [18]:
train.memory_usage(deep=True).sum() * 1e-6 # MB

339.364195

## Take date of only 2016

In [19]:
train.iloc[:2, [0,1]]

,Page,2015-07-01
0,2NE1_zh.wikipedia.org_all-access_spider,18
1,2PM_zh.wikipedia.org_all-access_spider,11


In [20]:
t1 = pd.Timestamp('2015-07-01')
t1

Timestamp('2015-07-01 00:00:00')

In [21]:
t2 = pd.Timestamp('2015-01-01')
t2-t1

Timedelta('-181 days +00:00:00')

In [22]:
train.iloc[:2, [0, 185, 185+365, -1]]

,Page,2016-01-01,2016-12-31,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,9,20,20
1,2PM_zh.wikipedia.org_all-access_spider,7,20,20


In [23]:
df = train.iloc[:, np.r_[0,185:185+366]]

df.head()

,Page,2016-01-01,2016-01-02,2016-01-03,2016-01-04,2016-01-05,2016-01-06,2016-01-07,2016-01-08,2016-01-09,2016-01-10,2016-01-11,2016-01-12,2016-01-13,2016-01-14,2016-01-15,2016-01-16,2016-01-17,2016-01-18,2016-01-19,2016-01-20,2016-01-21,2016-01-22,2016-01-23,2016-01-24,2016-01-25,2016-01-26,2016-01-27,2016-01-28,2016-01-29,2016-01-30,2016-01-31,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,2016-02-10,2016-02-11,2016-02-12,2016-02-13,2016-02-14,2016-02-15,2016-02-16,2016-02-17,2016-02-18,...,2016-11-12,2016-11-13,2016-11-14,2016-11-15,2016-11-16,2016-11-17,2016-11-18,2016-11-19,2016-11-20,2016-11-21,2016-11-22,2016-11-23,2016-11-24,2016-11-25,2016-11-26,2016-11-27,2016-11-28,2016-11-29,2016-11-30,2016-12-01,2016-12-02,2016-12-03,2016-12-04,2016-12-05,2016-12-06,2016-12-07,2016-12-08,2016-12-09,2016-12-10,2016-12-11,2016-12-12,2016-12-13,2016-12-14,2016-12-15,2016-12-16,2016-12-17,2016-12-18,2016-12-19,2016-12-20,2016-12-21,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,9,16,6,19,20,19,22,30,14,16,22,15,15,26,16,13,27,18,13,32,31,16,38,18,9,14,10,24,8,15,18,10,23,17,11,26,14,8,12,9,11,34,17,29,11,9,14,21,12,...,13,8,15,14,12,6,11,10,42,21,24,14,11,204,14,45,33,28,18,14,47,15,14,18,20,14,16,14,20,60,22,15,17,19,18,21,21,47,65,17,32,63,15,26,14,20,22,19,18,20
1,2PM_zh.wikipedia.org_all-access_spider,7,15,14,14,11,13,12,12,24,15,38,18,26,15,12,14,40,19,13,39,19,16,19,11,76,14,19,26,19,17,30,17,17,17,19,11,175,10,5,12,7,12,14,19,11,19,17,15,19,...,12,11,14,28,23,20,9,12,11,14,14,15,15,11,20,13,19,621,57,17,23,19,21,47,28,22,22,65,27,17,17,13,9,18,22,17,15,22,23,19,17,42,28,15,9,30,52,45,26,20
2,3C_zh.wikipedia.org_all-access_spider,2,0,3,3,3,4,4,8,3,5,8,1,4,0,3,6,3,1,3,3,3,1,3,8,4,3,2,5,6,3,6,5,6,7,3,1,5,1,2,0,1,4,3,3,9,4,7,5,10,...,6,4,2,4,6,5,4,4,3,3,9,3,5,4,0,1,4,5,8,8,1,1,2,5,3,3,3,7,3,9,8,3,210,5,4,6,2,2,4,3,3,1,1,7,4,4,6,3,4,17
3,4minute_zh.wikipedia.org_all-access_spider,7,7,11,7,14,9,21,9,10,13,10,13,16,8,10,7,13,18,8,50,8,33,6,22,9,84,28,11,7,14,16,49,71,29,22,6,34,16,14,9,12,24,18,8,26,8,8,13,21,...,38,13,14,17,26,14,10,9,23,15,7,10,7,10,14,17,11,9,11,5,10,8,17,13,23,40,16,17,41,17,8,9,18,12,12,18,13,18,23,10,32,10,26,27,16,11,17,19,10,11
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,13,11,8,6,10,14,6,9,6,16,14,13,15,14,16,9,178,64,12,10,11,6,8,7,9,8,5,11,8,4,15,5,8,8,6,7,15,4,11,7,48,9,25,13,3,11,27,13,36,10


In [24]:
df = df.sample(frac=0.1,random_state=random_state)

## Melt the data

In [25]:
df = df.melt(id_vars=['Page'],var_name='date',value_name='visits')

print(df.shape)
df.head()

(5309196, 3)


,Page,date,visits
0,Sean_Connery_en.wikipedia.org_desktop_all-agents,2016-01-01,4872
1,Tableau_des_médailles_des_Jeux_olympiques_d'été_de_2008_fr.wikipedia.org_desktop_all-agents,2016-01-01,6
2,The_Undertaker_fr.wikipedia.org_mobile-web_all-agents,2016-01-01,469
3,Category:Outdoor_sex_commons.wikimedia.org_all-access_all-agents,2016-01-01,142
4,Камызяк_ru.wikipedia.org_all-access_all-agents,2016-01-01,6692


In [26]:
df['date'] = pd.to_datetime(df['date'])

## Create date time columns

In [27]:
show_method_attributes(df['date'].dt)

,0,1,2,3,4,5,6
0,ceil,days_in_month,is_month_end,microsecond,quarter,to_period,week
1,date,daysinmonth,is_month_start,minute,round,to_pydatetime,weekday
2,day,floor,is_quarter_end,month,second,tz,weekday_name
3,day_name,freq,is_quarter_start,month_name,strftime,tz_convert,weekofyear
4,dayofweek,hour,is_year_end,nanosecond,timetz,tz_localize,year
5,dayofyear,is_leap_year,is_year_start,normalize,,,


In [28]:
%%time

# we can make these values categorical to reduce memory,
# but later for modelling, we need numpy array.
df['year'] = df['date'].dt.year # yyyy
df['month'] = df['date'].dt.month # 1 to 12
df['day'] = df['date'].dt.day # 1 to 31

df['quarter'] = df['date'].dt.quarter # 1 to 4
df['dayofweek'] = df['date'].dt.dayofweek # 0 to 6
df['dayofyear'] = df['date'].dt.dayofyear # 1 to 366 (leap year)

df['day_name'] = df['date'].dt.day_name() # Monday
df['month_name'] = df['date'].dt.month_name() # January

df['weekend'] = ((df['date'].dt.dayofweek) // 5 == 1)
df['weekday'] = ((df['date'].dt.dayofweek) // 5 != 1)

CPU times: user 4.97 s, sys: 290 ms, total: 5.26 s
Wall time: 5.6 s


In [29]:
df.head(20)

,Page,date,visits,year,month,day,quarter,dayofweek,dayofyear,day_name,month_name,weekend,weekday
0,Sean_Connery_en.wikipedia.org_desktop_all-agents,2016-01-01,4872,2016,1,1,1,4,1,Friday,January,False,True
1,Tableau_des_médailles_des_Jeux_olympiques_d'été_de_2008_fr.wikipedia.org_desktop_all-agents,2016-01-01,6,2016,1,1,1,4,1,Friday,January,False,True
2,The_Undertaker_fr.wikipedia.org_mobile-web_all-agents,2016-01-01,469,2016,1,1,1,4,1,Friday,January,False,True
3,Category:Outdoor_sex_commons.wikimedia.org_all-access_all-agents,2016-01-01,142,2016,1,1,1,4,1,Friday,January,False,True
4,Камызяк_ru.wikipedia.org_all-access_all-agents,2016-01-01,6692,2016,1,1,1,4,1,Friday,January,False,True
5,File:PioneerSodHouse-WheatRidgeCO.jpg_commons.wikimedia.org_desktop_all-agents,2016-01-01,0,2016,1,1,1,4,1,Friday,January,False,True
6,Международная_космическая_станция_ru.wikipedia.org_all-access_spider,2016-01-01,33,2016,1,1,1,4,1,Friday,January,False,True
7,Volleyball_at_the_2016_Summer_Olympics_–_Men's_tournament_en.wikipedia.org_all-access_all-agents,2016-01-01,123,2016,1,1,1,4,1,Friday,January,False,True
8,Marianne_James_fr.wikipedia.org_all-access_all-agents,2016-01-01,424,2016,1,1,1,4,1,Friday,January,False,True
9,頑童MJ116_zh.wikipedia.org_desktop_all-agents,2016-01-01,254,2016,1,1,1,4,1,Friday,January,False,True


## Create aggregation columns

In [30]:
%%time
df['mean'] = df.groupby('Page')['visits'].transform('mean')
df['median'] = df.groupby('Page')['visits'].transform('median')

CPU times: user 3.04 s, sys: 328 ms, total: 3.37 s
Wall time: 3.6 s


# String Manipulation

Example of one page:
```
2NE1_zh.wikipedia.org_all-access_spider
name_project_access_agent

```

In [31]:
df['Page'].head(10)

0                                                    Sean_Connery_en.wikipedia.org_desktop_all-agents
1         Tableau_des_médailles_des_Jeux_olympiques_d'été_de_2008_fr.wikipedia.org_desktop_all-agents
2                                               The_Undertaker_fr.wikipedia.org_mobile-web_all-agents
3                                    Category:Outdoor_sex_commons.wikimedia.org_all-access_all-agents
4                                                      Камызяк_ru.wikipedia.org_all-access_all-agents
5                      File:PioneerSodHouse-WheatRidgeCO.jpg_commons.wikimedia.org_desktop_all-agents
6                                Международная_космическая_станция_ru.wikipedia.org_all-access_spider
7    Volleyball_at_the_2016_Summer_Olympics_–_Men's_tournament_en.wikipedia.org_all-access_all-agents
8                                               Marianne_James_fr.wikipedia.org_all-access_all-agents
9                                                         頑童MJ116_zh.wikipedia.org

In [32]:
regex = ( r'(.+)_'  # name
          r'(.+)_'  # project
          r'(.+)_' # access
          r'(.+)'  # agent
        )

lang_map ={'en':'English','ja':'Japanese','de':'German',
        'www':'Media','fr':'French','zh':'Chinese',
        'ru':'Russian','es':'Spanish','commons': 'Media'
       }

# another way
#
# df['agent'] = df['page'].str.split('_').str[-1]
# df['access'] = df['page'].str.split('_').str[-2]
# df['project'] = df['page'].str.split('_').str[-3]
# df['name'] = df['page'].str.split('_').str[:-3].str.join('_')

def myfunc(df):
    df = df.copy()
    df[['name','project','access','agent']]  = df['Page'].str.extract(regex,expand=True)
    df['lang'] = df['project'].str.split('.').str[0]
    df['language'] = df['lang'].map(lang_map)

    return df

In [33]:
import dask.dataframe as dd
import gc

ddf = dd.from_pandas(df, npartitions=40)

def dask_apply(): 
    return ddf.map_partitions(myfunc).compute()

df = dask_apply()

del ddf
gc.collect()

df.head()

,Page,date,visits,year,month,day,quarter,dayofweek,dayofyear,day_name,month_name,weekend,weekday,mean,median,name,project,access,agent,lang,language
0,Sean_Connery_en.wikipedia.org_desktop_all-agents,2016-01-01,4872,2016,1,1,1,4,1,Friday,January,False,True,3.4e+03,2.6e+03,Sean_Connery,en.wikipedia.org,desktop,all-agents,en,English
1,Tableau_des_médailles_des_Jeux_olympiques_d'été_de_2008_fr.wikipedia.org_desktop_all-agents,2016-01-01,6,2016,1,1,1,4,1,Friday,January,False,True,1.7e+02,18,Tableau_des_médailles_des_Jeux_olympiques_d'été_de_2008,fr.wikipedia.org,desktop,all-agents,fr,French
2,The_Undertaker_fr.wikipedia.org_mobile-web_all-agents,2016-01-01,469,2016,1,1,1,4,1,Friday,January,False,True,4e+02,3.5e+02,The_Undertaker,fr.wikipedia.org,mobile-web,all-agents,fr,French
3,Category:Outdoor_sex_commons.wikimedia.org_all-access_all-agents,2016-01-01,142,2016,1,1,1,4,1,Friday,January,False,True,2.1e+02,1.9e+02,Category:Outdoor_sex,commons.wikimedia.org,all-access,all-agents,commons,Media
4,Камызяк_ru.wikipedia.org_all-access_all-agents,2016-01-01,6692,2016,1,1,1,4,1,Friday,January,False,True,9.1e+02,5.6e+02,Камызяк,ru.wikipedia.org,all-access,all-agents,ru,Russian


In [44]:
for x in ['project','agent','access','lang']:
    print(x)
    print(df[x].value_counts())
    print()

project
en.wikipedia.org         863028
ja.wikipedia.org         756156
de.wikipedia.org         688080
fr.wikipedia.org         637938
zh.wikipedia.org         637572
ru.wikipedia.org         566568
es.wikipedia.org         506544
commons.wikimedia.org    390888
www.mediawiki.org        262422
Name: project, dtype: int64

agent
all-agents    4050156
spider        1259040
Name: agent, dtype: int64

access
all-access    2705106
mobile-web    1331874
desktop       1272216
Name: access, dtype: int64

lang
en         863028
ja         756156
de         688080
fr         637938
zh         637572
ru         566568
es         506544
commons    390888
www        262422
Name: lang, dtype: int64



# Save the clean data

In [50]:
%%time
df.to_csv('../../data/wiki/processed/data_cleaned_2016_frac01.csv',index=False)

# 1.14 GB, cant be uploaded to github.

# CPU times: user 2min 15s, sys: 2.14 s, total: 2min 17s
# Wall time: 2min 36s

# pandas is faster to write file.

CPU times: user 2min 18s, sys: 1.86 s, total: 2min 20s
Wall time: 2min 30s


In [48]:
# %%time
# ddf = dd.from_pandas(df, npartitions=40)

# ddf.to_csv('../../data/wiki/processed/wikipedia_2016_frac01.csv',index=False)

# CPU times: user 2min 34s, sys: 3.74 s, total: 2min 38s
# Wall time: 2min 45s

CPU times: user 2min 34s, sys: 3.74 s, total: 2min 38s
Wall time: 2min 45s
